In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('data for project/turbines.csv')
data

,dt,MW
0,2011-01-01 00:00:00,3416.00
1,2011-01-01 00:15:00,4755.00
2,2011-01-01 00:30:00,4939.00
3,2011-01-01 00:45:00,4939.00
4,2011-01-01 01:00:00,4998.00
...,...,...
385561,2021-12-30 06:45:00,8140.31
385562,2021-12-30 07:00:00,8162.33
385563,2021-12-30 07:15:00,8405.11
385564,2021-12-30 07:30:00,8503.48


In [3]:
data.columns

Index(['dt', 'MW'], dtype='object')

In [4]:
nan_values = data.isna().sum()
nan_values

dt    0
MW    0
dtype: int64

In [5]:
#checking for duplicates.
duplicates = data.duplicated()
display(duplicates)
display(duplicates.nunique())

0         False
1         False
2         False
3         False
4         False
          ...  
385561    False
385562    False
385563    False
385564    False
385565    False
Length: 385566, dtype: bool

1

In [6]:
#now I want to check out the amount of MW produced on the first day and to check how the separation could be done
data['dt'] = pd.to_datetime(data['dt'])
# Filter rows where date is '2011-01-01'
first_day = data[data['dt'].dt.date == pd.to_datetime('2011-01-01').date()]
first_day

,dt,MW
0,2011-01-01 00:00:00,3416.0
1,2011-01-01 00:15:00,4755.0
2,2011-01-01 00:30:00,4939.0
3,2011-01-01 00:45:00,4939.0
4,2011-01-01 01:00:00,4998.0
...,...,...
91,2011-01-01 22:45:00,3614.0
92,2011-01-01 23:00:00,3639.0
93,2011-01-01 23:15:00,3484.0
94,2011-01-01 23:30:00,3576.0


In [7]:
#we have 96 rows with 15 minutes difference in each. (96 rows * 15 minutes and devided by 60 (minutes in hour) we get the amount of 24 hours.
96*15/60

24.0

In [8]:
#checking the amount of MW produced on this date to try to combine all days in my dataframy into one separate date with the sum of MW for a specific day
sum_of_MW = first_day['MW'].sum()
print("Sum of 'MW' on 2011-01-01:", sum_of_MW)

Sum of 'MW' on 2011-01-01: 518229.0


In [9]:
# Set the timestamp as the index
data.set_index('dt', inplace=True)

# Resample data to daily frequency and calculate summary statistics
data = data.resample('D').agg({
    'MW': ['sum']
})

# Rename columns for clarity
data.columns = ['Sum Value']
data

,Sum Value
dt,
2011-01-01,518229.00
2011-01-02,83077.00
2011-01-03,73095.00
2011-01-04,376606.00
2011-01-05,216599.00
...,...
2021-12-26,294921.35
2021-12-27,603998.08
2021-12-28,659450.45


In [10]:
data.columns

Index(['Sum Value'], dtype='object')

In [11]:
#now I want to recet index again to have have it as a separated column and rename it after
data.reset_index(inplace=True)
data.columns

Index(['dt', 'Sum Value'], dtype='object')

In [12]:
#renaming columns into the one that I want to have
data.rename(columns={'dt': 'date', 'Sum Value': 'windturbines_mw'}, inplace=True)

In [13]:
nan_values = data.isna().sum()
nan_values

date               0
windturbines_mw    0
dtype: int64

In [14]:
data.dtypes

date               datetime64[ns]
windturbines_mw           float64
dtype: object

In [15]:
#changing date column into an object
data['date'] = data['date'].astype(str)

In [16]:
data.dtypes

date                object
windturbines_mw    float64
dtype: object

In [17]:
data

,date,windturbines_mw
0,2011-01-01,518229.00
1,2011-01-02,83077.00
2,2011-01-03,73095.00
3,2011-01-04,376606.00
4,2011-01-05,216599.00
...,...,...
4012,2021-12-26,294921.35
4013,2021-12-27,603998.08
4014,2021-12-28,659450.45
4015,2021-12-29,241640.15


In [18]:
#data.to_csv('data for project/turbines_days_cleaned.csv', index=False) #saved after changing the file into a dayly format

In [19]:
hamburg = pd.read_csv('data for project/hamburg.csv')
hamburg

,date,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,2011-01-01,2.7,1.0,4.0,0.0,110.0,277.0,22.7,51.8,1013.4,252
1,2011-01-02,0.9,-0.8,2.8,0.0,80.0,284.0,12.2,26.3,1017.2,162
2,2011-01-03,-0.8,-2.8,0.9,0.0,70.0,257.0,9.0,20.9,1020.1,84
3,2011-01-04,0.8,-1.1,1.5,0.0,60.0,220.0,18.7,40.0,1016.0,0
4,2011-01-05,-2.3,-4.5,-0.6,0.6,50.0,175.0,19.1,37.4,1010.1,252
...,...,...,...,...,...,...,...,...,...,...,...
4012,2021-12-26,-6.0,-9.8,-1.8,0.1,0.0,104.0,10.8,31.3,1013.2,395
4013,2021-12-27,-2.6,-5.9,0.9,0.0,0.0,119.0,17.3,32.0,1004.7,0
4014,2021-12-28,1.8,0.8,2.7,2.2,0.0,124.0,15.8,32.8,995.9,0
4015,2021-12-29,3.1,1.4,6.2,0.6,0.0,180.0,5.8,19.1,1003.3,0


In [20]:
nan_values = hamburg.isna().sum()
nan_values

date    0
tavg    0
tmin    0
tmax    0
prcp    0
snow    0
wdir    0
wspd    0
wpgt    0
pres    0
tsun    0
dtype: int64

In [21]:
hamburg.dtypes

date     object
tavg    float64
tmin    float64
tmax    float64
prcp    float64
snow    float64
wdir    float64
wspd    float64
wpgt    float64
pres    float64
tsun      int64
dtype: object

In [22]:
cologne = pd.read_csv('data for project/cologne.csv')
cologne

,date,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,2011-01-01,1.4,0.1,2.8,1.5,90.0,260.0,7.9,24.5,1020.7,0
1,2011-01-02,0.9,-0.2,1.8,0.4,60.0,291.0,7.9,20.9,1022.7,0
2,2011-01-03,0.4,-0.4,2.0,1.4,50.0,257.0,5.8,15.1,1023.3,12
3,2011-01-04,-1.3,-4.2,0.1,0.0,60.0,150.0,8.3,22.7,1018.4,120
4,2011-01-05,-2.9,-4.3,1.3,6.4,60.0,131.0,16.9,34.9,1010.5,210
...,...,...,...,...,...,...,...,...,...,...,...
4012,2021-12-26,1.9,0.1,3.2,0.0,0.0,115.0,18.4,36.4,1006.1,0
4013,2021-12-27,5.6,2.8,8.0,5.1,0.0,120.0,16.2,27.7,1000.1,43
4014,2021-12-28,8.7,6.8,10.9,13.6,0.0,166.0,16.9,66.6,994.5,17
4015,2021-12-29,9.6,8.2,13.0,6.1,0.0,230.0,14.8,43.9,1006.0,19


In [23]:
nan_values = cologne.isna().sum()
nan_values

date    0
tavg    0
tmin    0
tmax    0
prcp    0
snow    0
wdir    0
wspd    0
wpgt    0
pres    0
tsun    0
dtype: int64

In [24]:
cologne.dtypes

date     object
tavg    float64
tmin    float64
tmax    float64
prcp    float64
snow    float64
wdir    float64
wspd    float64
wpgt    float64
pres    float64
tsun      int64
dtype: object

In [25]:
berlin = pd.read_csv('data for project/berlin.csv')
berlin

,date,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,2011-01-01,2.2,1.0,3.3,0.0,260.0,268.0,29.5,56.2,1011.7,18
1,2011-01-02,0.0,-3.1,1.5,0.0,150.0,275.0,16.2,37.8,1015.7,48
2,2011-01-03,-2.5,-5.3,-0.9,0.2,150.0,265.0,11.5,29.2,1019.8,0
3,2011-01-04,-2.8,-7.9,-0.7,0.0,150.0,202.0,9.7,24.5,1018.6,0
4,2011-01-05,-7.0,-10.5,-3.4,0.0,140.0,161.0,11.5,28.8,1015.1,342
...,...,...,...,...,...,...,...,...,...,...,...
4012,2021-12-26,-8.3,-12.2,-2.7,0.0,0.0,92.0,9.0,21.6,1014.6,369
4013,2021-12-27,-4.3,-9.9,-1.2,0.0,0.0,102.0,16.6,31.3,1008.3,15
4014,2021-12-28,-0.5,-4.2,3.0,2.5,0.0,119.0,16.6,31.3,1000.1,0
4015,2021-12-29,4.1,1.2,5.8,0.2,0.0,215.0,8.3,22.3,1004.0,0


In [26]:
nan_values = berlin.isna().sum()
nan_values

date    0
tavg    0
tmin    0
tmax    0
prcp    0
snow    0
wdir    0
wspd    0
wpgt    0
pres    0
tsun    0
dtype: int64

In [27]:
berlin.dtypes

date     object
tavg    float64
tmin    float64
tmax    float64
prcp    float64
snow    float64
wdir    float64
wspd    float64
wpgt    float64
pres    float64
tsun      int64
dtype: object

In [28]:
turbines = pd.read_csv('data for project/turbines_days_cleaned.csv')
turbines

,date,windturbines_mw
0,2011-01-01,518229.00
1,2011-01-02,83077.00
2,2011-01-03,73095.00
3,2011-01-04,376606.00
4,2011-01-05,216599.00
...,...,...
4012,2021-12-26,294921.35
4013,2021-12-27,603998.08
4014,2021-12-28,659450.45
4015,2021-12-29,241640.15


In [29]:
nan_values = turbines.isna().sum()
nan_values

date               0
windturbines_mw    0
dtype: int64

In [30]:
turbines.dtypes

date                object
windturbines_mw    float64
dtype: object

In [31]:
cologne.columns

Index(['date', 'tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt',
       'pres', 'tsun'],
      dtype='object')

In [32]:
# Merge the datasets on the date column
merged_data = pd.merge(hamburg, cologne, on='date', suffixes=('_hamburg', '_cologne'))
merged_data = pd.merge(merged_data, berlin, on='date')

In [33]:
merged_data

,date,tavg_hamburg,tmin_hamburg,tmax_hamburg,prcp_hamburg,snow_hamburg,wdir_hamburg,wspd_hamburg,wpgt_hamburg,pres_hamburg,...,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,2011-01-01,2.7,1.0,4.0,0.0,110.0,277.0,22.7,51.8,1013.4,...,2.2,1.0,3.3,0.0,260.0,268.0,29.5,56.2,1011.7,18
1,2011-01-02,0.9,-0.8,2.8,0.0,80.0,284.0,12.2,26.3,1017.2,...,0.0,-3.1,1.5,0.0,150.0,275.0,16.2,37.8,1015.7,48
2,2011-01-03,-0.8,-2.8,0.9,0.0,70.0,257.0,9.0,20.9,1020.1,...,-2.5,-5.3,-0.9,0.2,150.0,265.0,11.5,29.2,1019.8,0
3,2011-01-04,0.8,-1.1,1.5,0.0,60.0,220.0,18.7,40.0,1016.0,...,-2.8,-7.9,-0.7,0.0,150.0,202.0,9.7,24.5,1018.6,0
4,2011-01-05,-2.3,-4.5,-0.6,0.6,50.0,175.0,19.1,37.4,1010.1,...,-7.0,-10.5,-3.4,0.0,140.0,161.0,11.5,28.8,1015.1,342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4012,2021-12-26,-6.0,-9.8,-1.8,0.1,0.0,104.0,10.8,31.3,1013.2,...,-8.3,-12.2,-2.7,0.0,0.0,92.0,9.0,21.6,1014.6,369
4013,2021-12-27,-2.6,-5.9,0.9,0.0,0.0,119.0,17.3,32.0,1004.7,...,-4.3,-9.9,-1.2,0.0,0.0,102.0,16.6,31.3,1008.3,15
4014,2021-12-28,1.8,0.8,2.7,2.2,0.0,124.0,15.8,32.8,995.9,...,-0.5,-4.2,3.0,2.5,0.0,119.0,16.6,31.3,1000.1,0
4015,2021-12-29,3.1,1.4,6.2,0.6,0.0,180.0,5.8,19.1,1003.3,...,4.1,1.2,5.8,0.2,0.0,215.0,8.3,22.3,1004.0,0


In [34]:
# Select the columns that need averaging
weather_columns = ['tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt','pres', 'tsun']

# Calculate the mean for each weather statistic
for col in weather_columns:
    merged_data[col] = merged_data[[f'{col}_hamburg', f'{col}_cologne', col]].mean(axis=1)

# Select only the date and averaged columns
final_data = merged_data[['date'] + weather_columns]

In [35]:
final_data

,date,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,2011-01-01,2.100000,0.700000,3.366667,0.500000,153.333333,268.333333,20.033333,44.166667,1015.266667,90.000000
1,2011-01-02,0.600000,-1.366667,2.033333,0.133333,96.666667,283.333333,12.100000,28.333333,1018.533333,70.000000
2,2011-01-03,-0.966667,-2.833333,0.666667,0.533333,90.000000,259.666667,8.766667,21.733333,1021.066667,32.000000
3,2011-01-04,-1.100000,-4.400000,0.300000,0.000000,90.000000,190.666667,12.233333,29.066667,1017.666667,40.000000
4,2011-01-05,-4.066667,-6.433333,-0.900000,2.333333,83.333333,155.666667,15.833333,33.700000,1011.900000,268.000000
...,...,...,...,...,...,...,...,...,...,...,...
4012,2021-12-26,-4.133333,-7.300000,-0.433333,0.033333,0.000000,103.666667,12.733333,29.766667,1011.300000,254.666667
4013,2021-12-27,-0.433333,-4.333333,2.566667,1.700000,0.000000,113.666667,16.700000,30.333333,1004.366667,19.333333
4014,2021-12-28,3.333333,1.133333,5.533333,6.100000,0.000000,136.333333,16.433333,43.566667,996.833333,5.666667
4015,2021-12-29,5.600000,3.600000,8.333333,2.300000,0.000000,208.333333,9.633333,28.433333,1004.433333,6.333333


In [36]:
#it already looks fine, but I need to round the values up to one decimal and leave the column named tsun as the full number

In [37]:
#rounding all the columns up to 1 decimal, as it was originally in the dataframes and changing the type to int64 from float64 (same as it was for now)
columns_to_round = ['tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres']

final_data.loc[:, columns_to_round] = final_data.loc[:, columns_to_round].round(1)
final_data['tsun'] = final_data['tsun'].round().astype('int64')

C:\Users\lilly\AppData\Local\Temp\ipykernel_18452\3031977311.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['tsun'] = final_data['tsun'].round().astype('int64')


In [38]:
final_data

,date,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,2011-01-01,2.1,0.7,3.4,0.5,153.3,268.3,20.0,44.2,1015.3,90
1,2011-01-02,0.6,-1.4,2.0,0.1,96.7,283.3,12.1,28.3,1018.5,70
2,2011-01-03,-1.0,-2.8,0.7,0.5,90.0,259.7,8.8,21.7,1021.1,32
3,2011-01-04,-1.1,-4.4,0.3,0.0,90.0,190.7,12.2,29.1,1017.7,40
4,2011-01-05,-4.1,-6.4,-0.9,2.3,83.3,155.7,15.8,33.7,1011.9,268
...,...,...,...,...,...,...,...,...,...,...,...
4012,2021-12-26,-4.1,-7.3,-0.4,0.0,0.0,103.7,12.7,29.8,1011.3,255
4013,2021-12-27,-0.4,-4.3,2.6,1.7,0.0,113.7,16.7,30.3,1004.4,19
4014,2021-12-28,3.3,1.1,5.5,6.1,0.0,136.3,16.4,43.6,996.8,6
4015,2021-12-29,5.6,3.6,8.3,2.3,0.0,208.3,9.6,28.4,1004.4,6


In [39]:
final_data.dtypes

date     object
tavg    float64
tmin    float64
tmax    float64
prcp    float64
snow    float64
wdir    float64
wspd    float64
wpgt    float64
pres    float64
tsun      int64
dtype: object

In [40]:
nan_values = final_data.isna().sum()
nan_values

date    0
tavg    0
tmin    0
tmax    0
prcp    0
snow    0
wdir    0
wspd    0
wpgt    0
pres    0
tsun    0
dtype: int64

In [41]:
#checking for duplicates before merging with MW set.
duplicates = final_data.duplicated()
display(duplicates)
display(duplicates.nunique())

0       False
1       False
2       False
3       False
4       False
        ...  
4012    False
4013    False
4014    False
4015    False
4016    False
Length: 4017, dtype: bool

1

In [42]:
merged_data = pd.merge(final_data, turbines, on='date')

In [43]:
merged_data

,date,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,windturbines_mw
0,2011-01-01,2.1,0.7,3.4,0.5,153.3,268.3,20.0,44.2,1015.3,90,518229.00
1,2011-01-02,0.6,-1.4,2.0,0.1,96.7,283.3,12.1,28.3,1018.5,70,83077.00
2,2011-01-03,-1.0,-2.8,0.7,0.5,90.0,259.7,8.8,21.7,1021.1,32,73095.00
3,2011-01-04,-1.1,-4.4,0.3,0.0,90.0,190.7,12.2,29.1,1017.7,40,376606.00
4,2011-01-05,-4.1,-6.4,-0.9,2.3,83.3,155.7,15.8,33.7,1011.9,268,216599.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4012,2021-12-26,-4.1,-7.3,-0.4,0.0,0.0,103.7,12.7,29.8,1011.3,255,294921.35
4013,2021-12-27,-0.4,-4.3,2.6,1.7,0.0,113.7,16.7,30.3,1004.4,19,603998.08
4014,2021-12-28,3.3,1.1,5.5,6.1,0.0,136.3,16.4,43.6,996.8,6,659450.45
4015,2021-12-29,5.6,3.6,8.3,2.3,0.0,208.3,9.6,28.4,1004.4,6,241640.15


In [44]:
#final check before saving the data for duplicates in NaN values.
nan_values = merged_data.isna().sum()
nan_values

date               0
tavg               0
tmin               0
tmax               0
prcp               0
snow               0
wdir               0
wspd               0
wpgt               0
pres               0
tsun               0
windturbines_mw    0
dtype: int64

In [45]:
duplicates = merged_data.duplicated()
display(duplicates)
display(duplicates.nunique())

0       False
1       False
2       False
3       False
4       False
        ...  
4012    False
4013    False
4014    False
4015    False
4016    False
Length: 4017, dtype: bool

1

In [46]:
merged_data.dtypes

date                object
tavg               float64
tmin               float64
tmax               float64
prcp               float64
snow               float64
wdir               float64
wspd               float64
wpgt               float64
pres               float64
tsun                 int64
windturbines_mw    float64
dtype: object

In [47]:
#merged_data.to_csv('data for project/merged_with_weather.csv', index=False)